# 第九章 NPL 自然语言处理和分词

In [1]:
#hide
from fastbook import *
from IPython.display import display,HTML

NPL （Natural Language Processing）自然语言处理。

## 文本预处理
首先我们要做的是预测文本的下一个单词，在建立神经网络模型之前，对于文本中的单词，预处理主要有以下步骤：
    
1. 将所有出现的单词编译成可编程变量，将其列到一个列表中（这个list被成为vocab）
2. 给每个单词附上独特的编号
3. 建立一个嵌入矩阵来放这些单词，每一行对应一个vocab
4. 使用这些矩阵作为神经网络的第一层

对于文本数据，首先我们要把要预测的文本变成一个长长的字符串，然后模型的自变量应该是这串长长字符串的第一个单词到最后第二个单词，因变量是第二个单词到最后一个单词。

- 标记化：将文本转换为单词（或字符或子字符串，取决于模型的粒度）列表
- 数字化：列出出现的所有唯一词（词汇），并通过在词汇中查找其索引将每个单词转换为数字
- 语言模型数据加载器的创建：fastai提供了一个“ LMDataLoader”类，该类自动处理创建因变量而该因变量偏离自变量一个标记的情况。它还处理一些重要的细节，例如，如何以因变量和自变量按需保持其结构的方式对训练数据进行混洗
- 语言模型的创建：我们需要一种特殊的模型，该模型可以完成我们之前从未见过的事情：处理输入列表，可以任意大小。有很多方法可以做到这一点。在本章中，我们将使用“递归神经网络”（RNN）。我们将在<< chapter_nlp_dive >>中获得这些RNN的详细信息，但是现在，您可以将其视为另一个深度神经网络。

### 令牌化

在这个阶段主要处理的是如何将一个具有实际意义的词挑出来。遇到的问题会有很多，比如don’t是一个词还算是两个词呢？有些很长的化学名词和生物名词怎么办？如何处理中文和日语这样没有具体单词划分的语言？这些问题的解决要基于三个原则：

- 基于单词的分割：在词和词之间的空格出进行分割，同时也要将有语义不同的地方分割出来，比如说dont会分成do nt,这个情况下标点符号一般是分割的地方。_
- 基于子词的分割：根据最常出现的子字符串来进行分割，比如occasion可能会被分割成o c ca sion
- 基于字母的分割：单个字母单个字母进行分割

fastai没有做自己的分词器，但是做了一个接口可以调用不同的分词器


In [2]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [4]:
txt = files[0].open().read(); txt[:75]

'Once again Mr. Costner has dragged out a movie for far longer than necessar'

fastai默认的分词器是WorkTokenizer

In [5]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#187) ['Once','again','Mr.','Costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','Aside','from','the','terrific','sea','rescue','sequences',',','of','which','there','are','very','few','I'...]


我们可以从结果中看出一些分词的规律，比如说it's被分成了it和‘s，逗号被分成了单独的符号，

In [6]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

再来看一个例子，我们可以看出分词器知道U.S.即使有个'.',但是分词器也知道这个是一个单词，同时分词器也把1.00当成一个单词而不是两个单词。

同时fastai还有`Tokenizer`这个类，用来处理分词后的结果，使之可以被正常的使用

In [7]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


上面的是分词化的结果，除了原来的句子之外，还有一些奇怪的标识符，比如xxbos，xxmaj等，这些标识符都有自己的作用，比如说xxbos这个标识符意味着这个是这段文档的开头，xxmaj说明后面的单词的首字母需要大写，同时我们可以发现所有的大写都被转成了小写字符，原因如果不处理大写和小写的单词，会被程序识别成两个单词，但是事实上这两个单词是一个单词，只是大小写形式不同罢了。下面是一些标识符的意思。

- xxbos ::表示文本的开头 
- xxmaj ::表示下一个单词以大写字母开头
- xxunk ::表示下一个单词未知

所有的规则都可以在`defaults.text_proc_rules`被找到。


- fix_html ::用可读的版本替换特殊的HTML字符。
- replace_rep ：：用特殊的重复标记（xxrep）替换任何重复了3次或更多次的字符，然后重复该字符的次数，然后替换该字符；
- replace_wrep ::用特殊的单词重复标记（xxwrep），将重复了3次以上的单词替换为特殊标记，重复该单词的次数，然后替换该单词；
- spec_add_spaces ::在/和＃周围添加空格。
- rm_useless_spaces ::删除所有重复的空格字符。
- replace_all_copy ::小写所有大写字母的单词，并在其前面为所有大写字母（cap）添加一个特殊标记
- replace_maj ::小写一个大写单词，并在其前面添加一个用于大写的特殊标记（xxmaj）
- lowercase：:小写所有文本，并在开头（xxbos）添加一个特殊标记，

比如说这段文本

In [8]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

可以看到文档开头有xxbos的标识，'&copy'被转为了unicode，fast.ai的前面需要大写，所以有xxmaj的标识符，'xxrep','3','w',说明了www，INDEX被全部转为小写。

## 子词化subword

对于某些没有特别明确单词概念的语言来说（比如中文和日语），subword是一个更好的分词方式。这个步骤一般有两步：
- 找出文档中最常用的词群，将他们划为一个词
- 将这些词令牌化

In [9]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])
    

In [10]:
txts = L(o.open(encoding='utf-8').read() for o in files[:2000])
subword(1000)

'▁O n ce ▁again ▁M r . ▁Co st n er ▁has ▁d ra g g ed ▁out ▁a ▁movie ▁for ▁far ▁long er ▁than ▁ ne ce s s ary . ▁A side ▁from ▁the ▁ ter ri f'

分词效果不太好，Once被分成了O n ce，again倒是正常的被分类了。如果使用更大的参数效果会更好一点

In [11]:
subword(10000)

'▁On ce ▁again ▁Mr . ▁Costner ▁has ▁dragged ▁out ▁a ▁movie ▁for ▁far ▁longer ▁than ▁necessary . ▁A side ▁from ▁the ▁terrific ▁sea ▁rescue ▁sequences , ▁of ▁which ▁there ▁are ▁very ▁few ▁I ▁just ▁did ▁not ▁care ▁about ▁any ▁of'

虽然Once还是被分成了更加常见的on和ce，但是其他的单词的分词效果要好得多。参数越小，分词效果越过分，参数越大，分词效果越差。

完成分词之后，接下来的工作就是将分词完成的单词数字化，以便程序进行计算。在进行这项工作之前，我们可以看一下完成分词之后的频率。

In [12]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


In [13]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has'...]

In [14]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1968) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','i','in'...]"

这是一个60000词（这个大小是默认大小）的词频分析，可以看到经过令牌化之后，标识符的数量是最多的，其次是the和一些标点符号，再次是一些英语中的介词，of，to之类的。

In [15]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,  349,  183,    8, 1176,   10,    8, 1177,   60, 1455,   62,   12,   25,   28,  189,  957,   93,  958,   10])

经过序数化之后，我们就可以用一个数字来代替单词，2的意思是xxpad，8的意思是xxwrep，12的意思是句号.诸如此类。

 ## 创建批次

In [ ]:
完成数字化之后，为了使用SGD算法，需要建立mini-batch，我们先来看一下原文的令牌化之后的语料。

>  原文: In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while.

> 令牌化：xxbos xxmaj in this chapter , we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface . xxmaj first we will look at the processing steps necessary to convert text into numbers and how to customize it . xxmaj by doing this , we 'll have another example of the preprocessor used in the data block xxup api . \n xxmaj then we will study how we build a language model and train it for a while .

我们发现令牌化之后这段话有90个词，我们采用一行为15个，总共6行的方式来建立这样一个小批次。

In [16]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


看起来很整齐，如果这就是我们全部的数据，这样做当然没问题，但是对于一些大批量的数据，这样做问题就来了，即使我们使用更大一些的batch，比如64行，这样行内的元素可能有百万行。GPU就可能直呼受不住。所以我们需要更巧妙的方式。就像下面三个矩阵。

矩阵1

In [17]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


矩阵2

In [18]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


矩阵3

In [19]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


我们可以看到一个规律，这个6*15的大矩阵成功被我们分成了3个小矩阵，矩阵之间的规律是第一个矩阵的第一行->第二个矩阵的第一行->第三个矩阵的第一行->第一个矩阵的第二行->第二个矩阵的第二行->······，这样我们就可以将大的矩阵拆成很多个小的批次了。这么复杂的事情肯定库已经帮你做好了。

In [21]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)

In [22]:
x,y = first(dl)
x.shape,y.shape

((64, 72), (64, 72))

这个数据是前200个电影中的评论，将做好序数化的数据加载到`LMDataLoader`中就可以得到整理好的小batch了，默认的大小是64行*72列。

另外因变量是自变量的后一个单词组成的矩阵，比如说：

自变量：

In [23]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj once again xxmaj mr . xxmaj costner has dragged out a movie for far longer than necessary .'

因变量：

In [24]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj once again xxmaj mr . xxmaj costner has dragged out a movie for far longer than necessary . xxmaj'

## 使用fastai的方法

fastai会将这些工作都封装好了，当TextBlock被传入DataBlock的时候，令牌化和数字化的工作就会被自动的完成。

In [25]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

完成之后可以调用show_batch函数来查看批次

In [26]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj africa xxmaj screams , one of the least seen of abbott&costello 's films was an independent production that was released through xxmaj united xxmaj artists . xxmaj the thin plot has xxmaj hillary xxmaj brooke believing xxmaj costello has the map to a hidden territory that is rich with diamonds . xxmaj bud and xxmaj lou go to xxmaj africa at her behest with her two companions , the fighting xxmaj baer xxmaj brothers . xxmaj of course","xxmaj africa xxmaj screams , one of the least seen of abbott&costello 's films was an independent production that was released through xxmaj united xxmaj artists . xxmaj the thin plot has xxmaj hillary xxmaj brooke believing xxmaj costello has the map to a hidden territory that is rich with diamonds . xxmaj bud and xxmaj lou go to xxmaj africa at her behest with her two companions , the fighting xxmaj baer xxmaj brothers . xxmaj of course the"
1,"got the balls to make the xxmaj christians out to be the intolerant , xenophobic and reactionary half - wits . \n\n xxmaj moral xxmaj orel is still an interesting watch ( as long as it comes between superior shows on xxmaj adult xxmaj swim ) because it is a satire . xxmaj however , xxmaj it is more a satire on the people that make it rather then the people it is depicting . \n\n xxmaj if you ever","the balls to make the xxmaj christians out to be the intolerant , xenophobic and reactionary half - wits . \n\n xxmaj moral xxmaj orel is still an interesting watch ( as long as it comes between superior shows on xxmaj adult xxmaj swim ) because it is a satire . xxmaj however , xxmaj it is more a satire on the people that make it rather then the people it is depicting . \n\n xxmaj if you ever want"


接下来要做的就是对于模型进行微调了

In [27]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,        #AWD_LSTM是一个经过预训练的模型，就和rnet34是一样的概念，drop_mult是
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)    #进行一轮微调
#电脑不行了跑不动了

In [ ]:
训练完成之后，可以将模型储存下来